## 장바구니 분석과 연관 규칙 ##

__장바구니 분석__

장바구니 분석은 A 상품을 구입하면 B 상품을 구매한다는 식의 상품 간 구매 관련성을 분석한다.

상품들의 구매 관계에 관한 규칙을 연관 규칙(Association Rule)이라고 한다.

__장바구니 분석을 위한 샘플 데이터 입력__

하나의 InvoiceNo는 여러 상품 정보를 가질 수 있으며, 동일한 InvoiceNo는 한 번의 거래에서 함께 구매되었다는 의미이다. 따라서 동일한 InvocieNo 세트가 하나의 트랜잭션(거래)이다.

구매 이력 데이터는 트랜잭션 데이터이다.

In [8]:
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import sklearn

%precision 3


'%.3f'

In [9]:
import requests, zipfile
from io import StringIO
import io

In [10]:
trans = pd.read_excel('Online Retail.xlsx', sheet_name='Online Retail')
trans.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


__데이터 정리와 확인__

InvoiceNo의 첫 번째 문자는 해당 트랜잭션의 상태를 나타낸다.

5가 정상적인데이터, C가 취소, A는 불명확한 데이터이다.

In [11]:
#IncoiceNo의 첫 번째 문자를 추출해 cancel_flg 생성
trans['cancel_flg'] = trans.InvoiceNo.map(lambda x:str(x)[0])

# cancel_flg의 그룹을 나누고 집계
trans.groupby('cancel_flg').size()

cancel_flg
5    532618
A         3
C      9288
dtype: int64

In [12]:
trans = trans[(trans.cancel_flg == '5') & (trans.CustomerID.notnull())]

__연관 규칙__



In [13]:
# StockCode별로 데이터 개수를 집계하고 상위 5개 출력

trans['StockCode'].value_counts().head(5)


85123A    2035
22423     1724
85099B    1618
84879     1408
47566     1397
Name: StockCode, dtype: int64

1. 지지도(support)

지지도는 어떤 상품이 함께 구매된 장바구니의 수 또는 해당 장바구니 수가 전체에서 차지하는 비율이다.

지지도는 상대적인 값이며 높은지 낮은지 일률적으로 말할수 없다.

일반적으로 지지도가 낮으면 유용하지 않은 규칙이기 때문에, 발견한 규칙을 활용할지 말지 판달할 때 지지도를 기준으로 삼는다.

In [14]:
#모든 InvocieNo를 추출하고 trans_all에 저장
trans_all = set(trans.InvoiceNo)

#상품 85123A를 구입한 구매 데이터를 trans_a에 저장
trans_a = set(trans[trans['StockCode']=='85123A'].InvoiceNo)
print(len(trans_a))

#상품 85099B를 구입한 구매 데이터를 trans_b에 저장
trans_b = set(trans[trans['StockCode']=='85099B'].InvoiceNo)
print(len(trans_b))

#상품 05123A와 85099B를 동시에 구입한 구매 데이터를 trans_ab에 저장
trans_ab = trans_a&trans_b
print(len(trans_ab))

1978
1600
252


In [17]:
# 두 상품을 포함하는 trans_ab의 수를 출력
print('두 상품을 모두 포함하는 장바구니 수: {}'.format(len(trans_ab)))
print('두 상품을 모두 포함하는 장바구니가 전체에서 차지하는 비율: {:.3f}'.
      format(len(trans_ab)/len(trans_all)))

두 상품을 모두 포함하는 장바구니 수: 252
두 상품을 모두 포함하는 장바구니가 전체에서 차지하는 비율: 0.014


In [19]:
print('상품 85123A를 포함하는 장바구니 수: {}'.format(len(trans_a)))
print('상품 85123A를 포함하는 장바구니가 전체에서 차지하는 비율: {:.3f}'.
      format(len(trans_a)/len(trans_all)))

상품 85123A를 포함하는 장바구니 수: 1978
상품 85123A를 포함하는 장바구니가 전체에서 차지하는 비율: 0.107


2. 신뢰도(confidence)

어떤 상품 A의 구입 횟수를 기준으로 상품 A와 상품 B를 동시에 구매하는 비율이 어느 정도인지 나타낸다.

신뢰도가 높으면 함께 판매될 가능성이 높은 상품이므로연결판매를 계획한다면 신뢰도를 기준으로행사 상품을 선택하는 방식으로 연관 규칙을 활용할 수 있다.

단, 신뢰도만으로는 불충분하기 때문에 일반적으로 리프트 값도 함께 고려한다.

In [21]:
#상품 85123A를 구입하면 상품 85099B도 구입한다는 규칙의 신뢰도
print('신뢰도: {:.3f}'.format(len(trans_ab)/len(trans_a)))

신뢰도: 0.127


In [22]:
#상품 855099B를 구입하면 상품 85123A도 구입한다는 규칙의 신뢰도
print('신뢰도: {:.3f}'.format(len(trans_ab)/len(trans_b)))

신뢰도: 0.158


3. 리프트 값(lift)

상품 A를 구입하면 상품 B도 구매한다는 연관 규칙에서 리프트 값은 규칙의 신뢰도를 상품 B의 지지도로 나눈 값이다.

즉, 전체 장바구니에서 상품 B의 구매율과 상품 A와 상품 B를 함께 구매한 비율을 비교한 수치가 리프트 값이다.

리프트 값이 1.0보다 크면 같이 판매되기 쉽고 1.0보다 작으면 같이 판매되기 어렵다고 해석할 수  있다.

In [23]:
# 전체 장바구니에서 차지하는 상품 B의 구매 비율계산
support_b = len(trans_b) / len(trans_all)

# 상품 A를 구매하고 상품 B도 구매한 비율 계산
confidence = len(trans_ab) / len(trans_a)

# 리프트 값 계산
lift = confidence / support_b
print('lift: {:.3f}'.format(lift))

lift: 1.476
